In [ ]:
# %cd ..
!unzip Dataset.zip
!rm Dataset.zip
!unzip ValDataset.zip
!rm ValDataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: content/Dataset/Masks/54334.jpg  
  inflating: content/Dataset/Masks/46002.jpg  
  inflating: content/Dataset/Masks/506470.jpg  
  inflating: content/Dataset/Masks/84594.jpg  
  inflating: content/Dataset/Masks/108708.jpg  
  inflating: content/Dataset/Masks/76846.jpg  
  inflating: content/Dataset/Masks/152994.jpg  
  inflating: content/Dataset/Masks/423892.jpg  
  inflating: content/Dataset/Masks/202225.jpg  
  inflating: content/Dataset/Masks/538596.jpg  
  inflating: content/Dataset/Masks/275668.jpg  
  inflating: content/Dataset/Masks/395504.jpg  
  inflating: content/Dataset/Masks/259572.jpg  
  inflating: content/Dataset/Masks/485909.jpg  
  inflating: content/Dataset/Masks/186112.jpg  
  inflating: content/Dataset/Masks/557685.jpg  
  inflating: content/Dataset/Masks/554217.jpg  
  inflating: content/Dataset/Masks/78400.jpg  
   creating: content/Dataset/Images/
  inflating: content/Dataset/Images/316400.jpg  
  in

In [ ]:
!rm /content/content/Dataset/Images/111109.jpg
!rm /content/content/Dataset/Masks/111109.jpg

In [ ]:
import os
import time
from glob import glob

import torch
from torch.utils.data import DataLoader
import torch.nn as nn

from data import DriveDataset
from model import build_unet
from u_loss import DiceLoss, DiceBCELoss
from utils import seeding, create_dir, epoch_time

In [ ]:
def train(model, loader, optimizer, loss_fn, device):
    epoch_loss = 0.0

    model.train()
    for x, y in loader:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss

In [ ]:
device = torch.device('cuda')
print(device)

cuda


In [ ]:

""" Seeding """
seeding(42)

""" Directories """
#create_dir("files")

""" Load dataset """
train_x = sorted(glob("/content/content/Dataset/Images/*"))
train_y = sorted(glob("/content/content/Dataset/Masks/*"))

valid_x = sorted(glob("/content/content/ValDataset/Images/*"))
valid_y = sorted(glob("/content/content/ValDataset/Masks/*"))

data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
print(data_str)

""" Hyperparameters """
H = 256
W = 256
size = (H, W)
batch_size = 16
num_epochs = 30
lr = 1e-5
checkpoint_path = "/content/checkpoint-2.pth"

""" Dataset and loader """
train_dataset = DriveDataset(train_x, train_y)
valid_dataset = DriveDataset(valid_x, valid_y)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2
)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

device = torch.device('cuda')   ## GTX 1060 6GB
model = build_unet()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)
    
loss_fn = DiceBCELoss()

""" Training the model """
best_valid_loss = float("inf")
model.load_state_dict(torch.load(checkpoint_path))
for epoch in range(num_epochs):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, loss_fn, device)
    valid_loss = evaluate(model, valid_loader, loss_fn, device)

    """ Saving the model """
    if valid_loss < best_valid_loss:
        data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
        print(data_str)

        best_valid_loss = valid_loss
        torch.save(model.state_dict(), checkpoint_path)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
    data_str += f'\tTrain Loss: {train_loss:.3f}\n'
    data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
    print(data_str)


Dataset Size:
Train: 801 - Valid: 225

Valid loss improved from inf to 0.4487. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 01 | Epoch Time: 1m 25s
	Train Loss: 0.640
	 Val. Loss: 0.449

Valid loss improved from 0.4487 to 0.3721. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 02 | Epoch Time: 1m 19s
	Train Loss: 0.391
	 Val. Loss: 0.372

Valid loss improved from 0.3721 to 0.3217. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 03 | Epoch Time: 1m 19s
	Train Loss: 0.307
	 Val. Loss: 0.322

Valid loss improved from 0.3217 to 0.3093. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 04 | Epoch Time: 1m 19s
	Train Loss: 0.259
	 Val. Loss: 0.309

Valid loss improved from 0.3093 to 0.2776. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 05 | Epoch Time: 1m 19s
	Train Loss: 0.234
	 Val. Loss: 0.278

Valid loss improved from 0.2776 to 0.2690. Saving checkpoint: /content/checkpoint-2.pth
Epoch: 06 | Epoch Time: 1m 19s
	Train Loss: 0.218
	 Val. Loss: 0.269

Epoch: 07 | Epoc

KeyboardInterrupt: ignored

In [ ]:
import os, time
from operator import add
import numpy as np
from glob import glob
import cv2
from tqdm import tqdm
import imageio
import torch
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

from Seg.UNET.model import build_unet
from Seg.UNET.utils import create_dir, seeding

def calculate_metrics(y_true, y_pred):
    """ Ground truth """
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction """
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1) 
    mask = np.concatenate([mask, mask, mask], axis=-1)
    return mask


In [ ]:
""" Seeding """
seeding(42)

""" Folders """
#create_dir("results")

""" Load dataset """
test_x = sorted(glob("/content/content/ValDataset/Images/*"))
test_y = sorted(glob("/content/content/ValDataset/Masks/*"))

""" Hyperparameters """
H = 256
W = 256
size = (W, H)
checkpoint_path = "/content/checkpoint-2.pth"

""" Load the checkpoint """
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = build_unet()
model = model.to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()

metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
time_taken = []

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    """ Extract the name """
    name = x.split("/")[-1].split(".")[0]

    """ Reading image """
    image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
    ## image = cv2.resize(image, size)
    x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
    x = x/255.0
    x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
    x = x.astype(np.float32)
    x = torch.from_numpy(x)
    x = x.to(device)

    """ Reading mask """
    mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    ## mask = cv2.resize(mask, size)
    y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
    y = y/255.0
    y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
    y = y.astype(np.float32)
    y = torch.from_numpy(y)
    y = y.to(device)

    with torch.no_grad():
        """ Prediction and Calculating FPS """
        start_time = time.time()
        pred_y = model(x)
        pred_y = torch.sigmoid(pred_y)
        total_time = time.time() - start_time
        time_taken.append(total_time)


        score = calculate_metrics(y, pred_y)
        metrics_score = list(map(add, metrics_score, score))
        pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
        pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
        pred_y = pred_y > 0.5
        pred_y = np.array(pred_y, dtype=np.uint8)

    """ Saving masks """
    ori_mask = mask_parse(mask)
    pred_y = mask_parse(pred_y)
    line = np.ones((size[1], 10, 3)) * 128

    cat_images = np.concatenate(
        [image, line, ori_mask, line, pred_y * 255], axis=1
    )
    cv2.imwrite(f"/content/results/{name}.png", cat_images)

jaccard = metrics_score[0]/len(test_x)
f1 = metrics_score[1]/len(test_x)
recall = metrics_score[2]/len(test_x)
precision = metrics_score[3]/len(test_x)
acc = metrics_score[4]/len(test_x)
print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

fps = 1/np.mean(time_taken)
print("FPS: ", fps)

 26%|██▌       | 59/225 [00:05<00:16, 10.07it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 27%|██▋       | 61/225 [00:06<00:16, 10.09it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 47%|████▋     | 105/225 [00:10<00:10, 11.32it/s]/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start

Jaccard: 0.7934 - F1: 0.8646 - Recall: 0.8637 - Precision: 0.8857 - Acc: 0.9894
FPS:  145.49383378412878


In [ ]:
!rm -r /content/results/

In [ ]:
!zip -r /content/results_basemodel.zip /content/results

  adding: content/results/ (stored 0%)
  adding: content/results/482917.png (deflated 5%)
  adding: content/results/149222.png (deflated 3%)
  adding: content/results/464476.png (deflated 7%)
  adding: content/results/555009.png (deflated 3%)
  adding: content/results/166521.png (deflated 7%)
  adding: content/results/296231.png (deflated 4%)
  adding: content/results/442323.png (deflated 7%)
  adding: content/results/218439.png (deflated 3%)
  adding: content/results/520707.png (deflated 3%)
  adding: content/results/448076.png (deflated 4%)
  adding: content/results/131131.png (deflated 7%)
  adding: content/results/246308.png (deflated 5%)
  adding: content/results/198641.png (deflated 5%)
  adding: content/results/38576.png (deflated 16%)
  adding: content/results/301421.png (deflated 5%)
  adding: content/results/275749.png (deflated 8%)
  adding: content/results/540962.png (deflated 5%)
  adding: content/results/201418.png (deflated 4%)
  adding: content/results/319935.png (defla

In [ ]:
!unzip split_datset.zip

Archive:  split_datset.zip
   creating: content/Augmented_segmentation/
   creating: content/Augmented_segmentation/train/
   creating: content/Augmented_segmentation/train/masks/
  inflating: content/Augmented_segmentation/train/masks/47g_2.png  
  inflating: content/Augmented_segmentation/train/masks/21a_2.png  
  inflating: content/Augmented_segmentation/train/masks/23I_0.png  
  inflating: content/Augmented_segmentation/train/masks/37e_2.png  
  inflating: content/Augmented_segmentation/train/masks/57f_2.png  
  inflating: content/Augmented_segmentation/train/masks/25I_0.png  
  inflating: content/Augmented_segmentation/train/masks/29k_2.png  
  inflating: content/Augmented_segmentation/train/masks/35I_0.png  
  inflating: content/Augmented_segmentation/train/masks/15k2_0.png  
  inflating: content/Augmented_segmentation/train/masks/11k_2.png  
  inflating: content/Augmented_segmentation/train/masks/13f_0.png  
  inflating: content/Augmented_segmentation/train/masks/47g_1.png  
  i